In [ ]:
# Install requirements, run only once
!python -m venv venv
!source venv/bin/activate

!pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Connect to API, use API key from .env
from dotenv import load_dotenv, find_dotenv
from dune_client.client import DuneClient

load_dotenv(find_dotenv())
dune = DuneClient.from_env()

In [ ]:
# Fetch query results
UPDATE: bool = False

queries = [
    3794053,  # transactions: Count per type
    3855410,  # transactions: key-path vs. script-path
    3855891,  # transactions: inputs vs. outputs, w/o inscriptions
    3794022,  # transaction outputs: Taproot vs. other types
    3794171,  # transaction outputs: Percentage of Taproot outputs per month
]

if UPDATE:
    for query in queries:
        df = dune.get_latest_result_dataframe(query)
        df['block_date'] = pd.to_datetime(df['block_date'])
        df.to_csv(f'data/{query}.csv', index=False, date_format='%Y-%m-%d')
else:
    for idx, query in enumerate(queries):
        try:
            globals()[f"df{idx}"] = pd.read_csv(f'data/{query}.csv')
        except FileNotFoundError:
            pass

df1: pd.DataFrame
df2: pd.DataFrame
df3: pd.DataFrame
df4: pd.DataFrame
df5: pd.DataFrame
df6: pd.DataFrame
df7: pd.DataFrame


def get_layout(title: str, legend: str = None, x: str = None, y: str = None) -> go.Layout:
    return go.Layout(
        title=title,
        legend={'title': legend} if legend else None,
        xaxis={'title': x} if x else None,
        yaxis={'title': y} if y else None,
        template='presentation',
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        width=1920,
        height=600,
    )

# df1 = dune.get_latest_result_dataframe(QUERY1)
# df2 = dune.get_latest_result_dataframe(QUERY2)

# df1['block_date'] = pd.to_datetime(df1['block_date'])
# df1.to_csv(f'data/{QUERY1}.csv', index=False, date_format='%Y-%m-%d')

# df2['block_time'] = pd.to_datetime(df2['block_time'])
# df2.to_csv(f'data/{QUERY2}.csv', index=False,  date_format='%Y-%m-%d %H:%M:%S')

# df1 = pd.read_csv(f'data/{QUERY1}.csv')
# df2 = pd.read_csv(f'data/{QUERY2}.csv')

# Dune Dashboard
https://dune.com/gtn7c43fx/taproot-adoption-and-privacy/80f22449-0b58-4a06-9474-57db338b2cbc

## Number of transactions using Taproot
Transactions using Taproot over time

We want to distinguish between Taproot and other types of transactions. Additionally, because many if not most of Taproot transactions are coming from so-called ordinals/inscriptions (Bitcoin NFTs), we are mostly interested in Taproot transactions which weren't used for those special kind of transactions. This potentially provides more information about organic adoption which should give more insight into Bitcoin transaction privacy in general.


In [ ]:
x = df1['block_date']

fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=df1['tx_avg'],
            name='TX outputs',
        ),
        go.Scatter(
            x=x,
            y=df1['tx_taproot_avg'],
            name='TX outputs taproot',
            fill='tozeroy',
        ),
        go.Scatter(
            x=x,
            y=df1['tx_other_avg'],
            name='TX outputs other',
        ),
    ],
    layout=get_layout(
        'Transaction output types, 21 day moving average',
        'Count',
        'Block date',
        'Transactions per day'
    )
    # .update(
    #     annotations=[
    #         dict(text="Ordinals", x='2022-12-14'),
    #     ],
    # )
)
fig.write_image('data/taproot_usage.png')
fig.show()

## Number of transactions that are "ordinals"/have inscriptions vs. regular Taproot transactions

In [ ]:
QUERY5 = 3794295  # w/o ordinals, atomicals
QUERY6 = 3798977  # only ordinals, atomicals

# df5 = dune.get_latest_result_dataframe(QUERY5)
# df5['block_date'] = pd.to_datetime(df5['block_date'])
# df5.to_csv(f'data/{QUERY5}_1.csv', index=False, date_format='%Y-%m-%d')
# 
# df6 = dune.get_latest_result_dataframe(QUERY6)
# df6['block_date'] = pd.to_datetime(df6['block_date'])
# df6.to_csv(f'data/{QUERY6}_1.csv', index=False, date_format='%Y-%m-%d')


df5 = pd.read_csv(f'data/{QUERY5}_1.csv')
df6 = pd.read_csv(f'data/{QUERY6}_1.csv')

df6 = pd.merge(df5, df6, on='block_date', how='left')

df6['tx_avg_z'] = df6['tx_avg_x'] + df6['tx_avg_y']
df6['tx_taproot_avg_z'] = df6['tx_taproot_avg_x'] + df6['tx_taproot_avg_y']

In [ ]:
x = df5['block_date']

fig = go.Figure(
    data=[
        # go.Scatter(
        #     x=x,
        #     y=df5['tx_avg'],
        #     name='TX w/o inscriptions',
        # ),
        # go.Scatter(
        #     x=x,
        #     y=df5['tx_other_avg'],
        #     name='TX w/o inscriptions, other',
        # ),
        # go.Scatter(
        #     x=x,
        #     y=df6['tx_avg_z'],  # / df6['tx_avg_z'].max() * 100,
        #     name='TX w/ inscriptions',
        #     # fill='tonexty',
        # ),
        # go.Scatter(
        #     x=x,
        #     y=df6['tx_avg_z']/ df6['tx_avg_z'] * 100,  # / df6['tx_avg_z'].max() * 100,
        #     yaxis='y2',
        #     name='TX all',
        #     # fill='tonexty',
        # ),
        # go.Scatter(
        #     x=x,
        #     y=df5['tx_avg'] / df6['tx_avg_z'] * 100,
        #     yaxis='y2',
        #     name='TX w/o inscriptions',
        #     # fill='tozeroy',
        # ),
        go.Scatter(
            x=x,
            y=df6['tx_taproot_avg_z'] / df6['tx_avg_z'] * 100,
            yaxis='y2',
            name='TX w/ inscriptions, taproot',
            # fill='tozeroy',
        ),
        go.Scatter(
            x=x,
            y=df5['tx_taproot_avg'] / df6['tx_avg_z'] * 100,
            yaxis='y2',
            name='TX w/o inscriptions, taproot',
            fill='tozeroy',
        ),
        # go.Scatter(
        #     x=x,
        #     y=df6['tx_taproot_avg_z'] / df6['tx_taproot_avg_z'].max() * 100,
        #     name='TX w/ inscriptions, taproot',
        # ),
        # go.Scatter(
        #     x=x,
        #     y=df6['tx_other_avg_y'],
        #     name='TX inscriptions, other',
        # ),
    ],
    layout=go.Layout(
        template='presentation',
        title='Taproot transactions w/o inscriptions, 21 day moving average',
        legend={'title': 'Count'},
        xaxis={'title': 'Block date'},
        yaxis={'title': 'Transactions per day'},
        width=1920,
        height=720,
        yaxis2={
            'overlaying': 'y',
            'side': 'right',
            'ticksuffix': '%',
        }
    )
)

fig.write_image('data/taproot_usage_without_inscriptions_percentage.png')
fig.show()